# Label each row with features required to run ANN notebooks

The new features are:
* The time left until next bus stop (seconds)
* The time it takes to travel the full segment (seconds)
* The time from the start of the journey to the start of the current segment (seconds)

In [21]:
import numpy as np
import pandas as pds
import datetime as dt
import time
from sklearn import preprocessing as pp

In [39]:
df = pds.read_pickle('train_203_compressed.pkl')
df.head()

18.409999847412102

In [9]:
#df = pds.read_csv('bus203_all.csv')
#df.head()

Ignore entries that are not `ObservedPositionEvent`

In [23]:
df = df[df['event'] == "ObservedPositionEvent"]

This leaves the indexes of rows untouched, reset.

In [24]:
df = df.reset_index().drop(columns=['index'])

Normalize speed and coordinates

In [25]:
mm = pp.MinMaxScaler()

df['latitude'] = mm.fit_transform(df.latitude.values.reshape(-1,1))
df['longitude'] = mm.fit_transform(df.longitude.values.reshape(-1,1))
df['speed'] = mm.fit_transform(df.speed.values.reshape(-1,1))

Convert `timestamp` to pandas datetime object.

In [26]:
df['timestamp'] = pds.to_datetime(df['timestamp']).dt.tz_localize("UTC").dt.tz_convert("Europe/Stockholm")

A triple loop as it seems, but it is only to finally group rows from each individual segment from every journey. Should be linear in time, as the innermost loop will do all executions and every row is visited once. This only took ~50% of my 8GB of RAM but took ~30 minutes to run.

In [27]:
time_left = pds.DataFrame(np.zeros(len(df.index)), columns=['time_left'])
segment_time = pds.DataFrame(np.zeros(len(df.index)), columns=['segment_time'])
# Time since journey start
tsjs = pds.DataFrame(np.zeros(len(df.index)), columns=['tsjs'])


t0 = time.time()

for j, df_j in df.groupby('journey_number'):
    journey_start = df_j['timestamp'].iloc[0]
    for k, df_s in df_j.groupby('segment_number'):
        end_time = df_s['timestamp'].iloc[-1]
        start_time = df_s['timestamp'].iloc[0]
        for idx, row in df_s.iterrows():
            # The subtraction returns timedelta between the two timestamp objects
            # and total seconds convert the pandas datetime object to seconds
            time_left.iloc[idx] = (end_time - row['timestamp']).total_seconds()
            segment_time.iloc[idx] = (end_time - start_time).total_seconds()
            tsjs.iloc[idx] = (start_time - journey_start).total_seconds()

elapsed = time.time() - t0
print("Data processed in", elapsed, " seconds")

Data processed in 835.0628294944763  seconds


Add new features to dataframe `df`

In [28]:
data = pds.concat([df, time_left,segment_time, tsjs], axis=1)
data.head()

,compressed,direction,event,journey_number,latitude,longitude,segment_number,speed,timestamp,time_left,segment_time,tsjs
0,1,129.852944,ObservedPositionEvent,1,0.325030,0.926476,1,0.051520,2018-02-16 04:48:46+01:00,65.0,65.0,0.0
1,1,142.963046,ObservedPositionEvent,1,0.329938,0.928692,1,0.210201,2018-02-16 04:49:05+01:00,46.0,65.0,0.0
2,0,287.399994,ObservedPositionEvent,1,0.326437,0.944423,1,0.276662,2018-02-16 04:49:17+01:00,34.0,65.0,0.0
3,0,289.790009,ObservedPositionEvent,1,0.322491,0.946723,1,0.398248,2018-02-16 04:49:18+01:00,33.0,65.0,0.0
4,0,288.200012,ObservedPositionEvent,1,0.320008,0.948639,1,0.427615,2018-02-16 04:49:19+01:00,32.0,65.0,0.0


Renaming some columns to keep them similar to the GP model for easier understanding.

In [29]:
data.rename(columns={'longitude': 'lon', 'latitude': 'lat', 'segment_number': 'seg', 'journey_number': 'journey'}, inplace=True)
data.head()

,compressed,direction,event,journey,lat,lon,seg,speed,timestamp,time_left,segment_time,tsjs
0,1,129.852944,ObservedPositionEvent,1,0.325030,0.926476,1,0.051520,2018-02-16 04:48:46+01:00,65.0,65.0,0.0
1,1,142.963046,ObservedPositionEvent,1,0.329938,0.928692,1,0.210201,2018-02-16 04:49:05+01:00,46.0,65.0,0.0
2,0,287.399994,ObservedPositionEvent,1,0.326437,0.944423,1,0.276662,2018-02-16 04:49:17+01:00,34.0,65.0,0.0
3,0,289.790009,ObservedPositionEvent,1,0.322491,0.946723,1,0.398248,2018-02-16 04:49:18+01:00,33.0,65.0,0.0
4,0,288.200012,ObservedPositionEvent,1,0.320008,0.948639,1,0.427615,2018-02-16 04:49:19+01:00,32.0,65.0,0.0


Add history entries for model 3

In [30]:
history_speed = pds.DataFrame(0, index=np.arange(len(data)), columns=['speed_one', 'speed_two', 'speed_three'])
history_pos = pds.DataFrame(0, index=np.arange(len(data)), columns=['lat_one', 'lon_one', 'lat_two', 'lon_two', 'lat_three', 'lon_three'])

Add columns for the three latest positions for each sample. Replace the first sample from each journey with itself.

In [31]:
for i, a in data.groupby('journey'):
    idxs = a.index

    history_pos.lat_one.iloc[idxs] = a.lat.shift(1)
    history_pos.lat_one.iloc[idxs[0]] = history_pos.lat_one.iloc[1]

    history_pos.lon_one.iloc[idxs] = a.lon.shift(1)
    history_pos.lon_one.iloc[idxs[0]] = history_pos.lon_one.iloc[1]

    history_pos.lat_two.iloc[idxs] = a.lat.shift(2)
    history_pos.lat_two.iloc[idxs[0]] = history_pos.lat_two.iloc[2]
    history_pos.lat_two.iloc[idxs[1]] = history_pos.lat_two.iloc[2]

    history_pos.lon_two.iloc[idxs] = a.lon.shift(2)
    history_pos.lon_two.iloc[idxs[0]] = history_pos.lon_two.iloc[2]
    history_pos.lon_two.iloc[idxs[1]] = history_pos.lon_two.iloc[2]

    history_pos.lat_three.iloc[idxs] = a.lat.shift(3)
    history_pos.lat_three.iloc[idxs[0]] = history_pos.lat_three.iloc[3]
    history_pos.lat_three.iloc[idxs[1]] = history_pos.lat_three.iloc[3]
    history_pos.lat_three.iloc[idxs[2]] = history_pos.lat_three.iloc[3]

    history_pos.lon_three.iloc[idxs] = a.lon.shift(3)
    history_pos.lon_three.iloc[idxs[0]] = history_pos.lon_three.iloc[3]
    history_pos.lon_three.iloc[idxs[1]] = history_pos.lon_three.iloc[3]
    history_pos.lon_three.iloc[idxs[2]] = history_pos.lon_three.iloc[3]

C:\Users\Elias\Anaconda3\envs\py35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


and speed

In [40]:
for i, a in data.groupby('journey'):
    idxs = a.index

    history_speed.speed_one.iloc[idxs] = a.lat.shift(1)
    history_speed.speed_one.iloc[idxs[0]] = history_speed.speed_one.iloc[1]

    history_speed.speed_two.iloc[idxs] = a.lat.shift(2)
    history_speed.speed_two.iloc[idxs[0]] = history_speed.speed_two.iloc[2]
    history_speed.speed_two.iloc[idxs[1]] = history_speed.speed_two.iloc[2]

    history_speed.speed_three.iloc[idxs] = a.lat.shift(3)
    history_speed.speed_three.iloc[idxs[0]] = history_speed.speed_three.iloc[3]
    history_speed.speed_three.iloc[idxs[1]] = history_speed.speed_three.iloc[3]
    history_speed.speed_three.iloc[idxs[2]] = history_speed.speed_three.iloc[3]

C:\Users\Elias\Anaconda3\envs\py35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Make sure there are no NaN:s produced by the shifting

In [41]:
history_pos.isna().sum()

lat_one      0
lon_one      0
lat_two      0
lon_two      0
lat_three    0
lon_three    0
dtype: int64

In [42]:
history_speed.isna().sum()

speed_one      0
speed_two      0
speed_three    0
dtype: int64

The two cells above should contain only zeros, otherwise, model training will fail!

In [43]:
new_data = pds.concat([history_pos, history_speed], axis=1)
data = pds.concat([data, new_data], axis=1)
data.head()

,compressed,direction,event,journey,lat,lon,seg,speed,timestamp,time_left,...,tsjs,lat_one,lon_one,lat_two,lon_two,lat_three,lon_three,speed_one,speed_two,speed_three
0,1,129.852944,ObservedPositionEvent,1,0.325030,0.926476,1,0.051520,2018-02-16 04:48:46+01:00,65.0,...,0.0,0.325030,0.926476,0.325030,0.926476,0.325030,0.926476,0.325030,0.325030,0.325030
1,1,142.963046,ObservedPositionEvent,1,0.329938,0.928692,1,0.210201,2018-02-16 04:49:05+01:00,46.0,...,0.0,0.325030,0.926476,0.325030,0.926476,0.325030,0.926476,0.325030,0.325030,0.325030
2,0,287.399994,ObservedPositionEvent,1,0.326437,0.944423,1,0.276662,2018-02-16 04:49:17+01:00,34.0,...,0.0,0.329938,0.928692,0.325030,0.926476,0.325030,0.926476,0.329938,0.325030,0.325030
3,0,289.790009,ObservedPositionEvent,1,0.322491,0.946723,1,0.398248,2018-02-16 04:49:18+01:00,33.0,...,0.0,0.326437,0.944423,0.329938,0.928692,0.325030,0.926476,0.326437,0.329938,0.325030
4,0,288.200012,ObservedPositionEvent,1,0.320008,0.948639,1,0.427615,2018-02-16 04:49:19+01:00,32.0,...,0.0,0.322491,0.946723,0.326437,0.944423,0.329938,0.928692,0.322491,0.326437,0.329938


... remove unwanted columns ...

In [44]:
data = data.drop(columns=['compressed', 'event'])
#data = data.drop(columns=['Unnamed: 0', 'event', 'vehicle_id', 'line', 'station'])
data.head()

,direction,journey,lat,lon,seg,speed,timestamp,time_left,segment_time,tsjs,lat_one,lon_one,lat_two,lon_two,lat_three,lon_three,speed_one,speed_two,speed_three
0,129.852944,1,0.325030,0.926476,1,0.051520,2018-02-16 04:48:46+01:00,65.0,65.0,0.0,0.325030,0.926476,0.325030,0.926476,0.325030,0.926476,0.325030,0.325030,0.325030
1,142.963046,1,0.329938,0.928692,1,0.210201,2018-02-16 04:49:05+01:00,46.0,65.0,0.0,0.325030,0.926476,0.325030,0.926476,0.325030,0.926476,0.325030,0.325030,0.325030
2,287.399994,1,0.326437,0.944423,1,0.276662,2018-02-16 04:49:17+01:00,34.0,65.0,0.0,0.329938,0.928692,0.325030,0.926476,0.325030,0.926476,0.329938,0.325030,0.325030
3,289.790009,1,0.322491,0.946723,1,0.398248,2018-02-16 04:49:18+01:00,33.0,65.0,0.0,0.326437,0.944423,0.329938,0.928692,0.325030,0.926476,0.326437,0.329938,0.325030
4,288.200012,1,0.320008,0.948639,1,0.427615,2018-02-16 04:49:19+01:00,32.0,65.0,0.0,0.322491,0.946723,0.326437,0.944423,0.329938,0.928692,0.322491,0.326437,0.329938


... and save it.

In [45]:
data.to_pickle('ANN_train_203_downsampled.pkl')